# Binary Formats

Last time, we worked directly with byte-level data.  Input and output of binary data is reasonably straightforward given a known structure.  It is common for commercial software to use [proprietary data formats](https://en.wikipedia.org/wiki/Proprietary_software#Proprietary_file_formats_and_protocols), and [hundreds of formats](https://en.wikipedia.org/wiki/List_of_file_formats) exist for a wide variety of applications.  In the scientific community, a variety of [formats](https://en.wikipedia.org/wiki/List_of_file_formats#Scientific_data_(data_exchange)) were developed to facilitate the exchange of large, scientific data sets, including the popular [HDF5 format](https://portal.hdfgroup.org/display/HDF5/HDF5).  Other formats have been developed specifically to support visualization of large data sets; of these, we'll explore the [VTK formats](https://vtk.org/wp-content/uploads/2015/04/file-formats.pdf).  Although the HDF5 and VTK formats have very different purposes, their specifications are well developed and serve as good examples for those navigating the use of such third-party formats.

## HDF5

Of the scientific-data  developed, perhaps the [Hierarchical Data Format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) is the most widely used.

The actual, on-file, binary format is rarely important to know, but the in-file, data structure is.  


That [structure is like a file directory](https://portal.hdfgroup.org/display/HDF5/Introduction+to+HDF5#IntroductiontoHDF5-hdf5desc):

```python
/                 # root GROUP
/SimOut/          ## GROUP containing simulation output
  data2darray     ## a DATASET, 2-D array of doubles
        DATATYPE: H5T_NATIVE_DOUBLE,
       DATASPACE: {rank:2, dim=(5,3)},
      PROPERTIES: {layout: contiguous},
      ATTRIBUTES: {name1: value1, name2: value2, ...}
            DATA: ...
  data3darray     ## a DATASET, 3-D array of integers
        DATATYPE: H5T_NATIVE_INT,
       DATASPACE: {rank:3, dim=(4,3,2)},
      PROPERTIES: {layout: contiguous},
      ATTRIBUTES: {...}
            DATA: ...
/Viz/             ## GROUP containing viz output
  metadata=
    {ATTRIBUTES: {...} # groups can have attributes
  image1          ## a DATASET, an image (2-D array?)
    ...
  image2  
     .... and so on
```

### Creating an HDF5 File

In [1]:
import h5py
import numpy as np
name =  'mytestfile.hdf5'
!rm mytestfile.hdf5 # makes sure it's not already there
fh = h5py.File('mytestfile.hdf5', 'w')
SimOut = fh.create_group('SimOut')
data2darray = SimOut.create_dataset("data2darray", data=np.random.randint(0, 10, (5, 4)))
data2darray.attrs["name1"] = "value1"
data2darray.attrs["name2"] = "value2"
data3darray = SimOut.create_dataset("data3darray", data=np.random.rand(4,3,2))
Viz = fh.create_group('Viz')
image1 = Viz.create_dataset("image1", data=[1,2,3])
image2 = Viz.create_dataset("image2", data=[2,3,4])
fh.close()

In [2]:
!h5dump mytestfile.hdf5

HDF5 "mytestfile.hdf5" {
GROUP "/" {
   GROUP "SimOut" {
      DATASET "data2darray" {
         DATATYPE  H5T_STD_I64LE
         DATASPACE  SIMPLE { ( 5, 4 ) / ( 5, 4 ) }
         DATA {
         (0,0): 4, 6, 2, 2,
         (1,0): 0, 7, 0, 6,
         (2,0): 6, 9, 0, 1,
         (3,0): 3, 2, 7, 3,
         (4,0): 1, 0, 4, 5
         }
         ATTRIBUTE "name1" {
            DATATYPE  H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            }
            DATASPACE  SCALAR
            DATA {
            (0): "value1"
            }
         }
         ATTRIBUTE "name2" {
            DATATYPE  H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            }
            DATASPACE  SCALAR
            DATA {
            (0): "value2"
            }
         }
      }
      DATASET

### Reading an HDF5 File 

Writing it out is easy enough, but how do we get that back in?

In [3]:
import h5py
import numpy as np
name =  'mytestfile.hdf5'
fh = h5py.File('mytestfile.hdf5', 'r')
d2da = fh.get("/SimOut/data2darray")
print("shape = ", d2da.shape)
print("data = ", d2da[()]) # or d2da.value, which is deprecated
fh.close()

shape =  (5, 4)
data =  [[4 6 2 2]
 [0 7 0 6]
 [6 9 0 1]
 [3 2 7 3]
 [1 0 4 5]]


The reason HDF5 is so useful is because this API is available in the core C/C++ language used to build HDF5 and several other languages (Python, MATLAB, Fortran, etc.).

## VTK

The Visualization Toolkit (VTK) defines a variety of formats for storing data in a way that is easy to load, inspect, and modify in visualization tools like [Paraview]() and [VisIt]().  

VTK defiens two broad classes of data formats: the [legacy](https://lorensen.github.io/VTKExamples/site/VTKFileFormats/#simple-legacy-formats) formats and the [XML](https://lorensen.github.io/VTKExamples/site/VTKFileFormats/#xml-file-formats) formats.  Despite being "legacy", these formats are still widely used.  The XML variants give a bit more flexibility for parallel file input and output, but the underlying data organization is quite similar.  Here, we'll explore the legacy formats.

### Making a VTK File In Text Format

The VTK formats are easy enough to write directly to ASCII-formatted text files.  For large data, the binary variants of these formats is preferred, so we'll start with the text version and then do the same in binary using the `vtk` module.

For some sample data, we'll evaluate the function $f(x,y,z) = xy^2\sin(z)$ on the grid $x,y,z \in [0, 1]$.  Here, we'll use 3, 4, and 5 equally-spaced divisions in each dimension.  We'll also define the 3-component gradient:

In [4]:
x = np.linspace(0, 1, 2)
y = np.linspace(0, 1, 3)
z = np.linspace(0, 1, 4)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
f = ((xx+1)*(yy+1)**2*np.sin(zz))
fx, fy, fz = np.gradient(f, x, y, z)
f = f.flatten()
grad_f = np.array([fx.flatten(), fy.flatten(), fz.flatten()]).T

The legacy formats all share a basic form, but the specifics differ enough that we can just dive in.  For the data we have, two legacy formats are applicable: the *Structured Grid* and *Rectilinear Grid* formats.  The former applies whenever the spacing is constant along an axis.  The latter lets us define points with variable spacing, and we'll use this for generality.  The basic format (with my `// comments`) is

```
# vtk DataFile Version 2.0 // required header
DATASET RECTILINEAR_GRID   // the data format
DIMENSIONS nx ny nz        // dimensions of the grid
X_COORDINATES nx dataType  // dataType here is double
x[0] x[1] ... x[nx-1]
Y_COORDINATES ny dataType
y[0] y[1] ... y[ny-1]
Z_COORDINATES nz dataType
z[0] z[1] ... z[nz-1]
SCALARS name_of_sdata dataType 
LOOKUP_TABLE tableName     // tableName can be set to default
val[0] 
val[1]
...
VECTOR name_of_vdata dataType 
LOOKUP_TABLE tableName     
val.x[0] val.y[0] val.z[0]
val.x[1] val.y[1] val.z[1]
...
```

In [5]:
def make_vtk_rectilinear_grid(x, y, z, scalar_vals, vector_vals):
    return f"""# vtk DataFile Version 2.0
some title
ASCII
DATASET RECTILINEAR_GRID
DIMENSIONS {len(x)} {len(y)} {len(z)}
X_COORDINATES {len(x)} double
 {np.array2string(x, floatmode="fixed")[1:-1]}
Y_COORDINATES {len(y)} double
 {np.array2string(y, floatmode="fixed")[1:-1]}
Z_COORDINATES {len(z)} double
 {np.array2string(z, floatmode="fixed")[1:-1]}
POINT_DATA {len(x)*len(y)*len(z)}
SCALARS scalar_data double 1
LOOKUP_TABLE default
 {np.array2string(scalar_vals,1,floatmode="fixed").replace('[','').replace(']','')}
VECTORS vector_data double
 {np.array2string(vector_vals,floatmode="fixed").replace('[','').replace(']','')}
"""

In [6]:
s = make_vtk_rectilinear_grid(x, y, z, f, grad_f)
fh = open("rectlin.vtk", "w")
fh.write(s)
fh.close()
print(s)

# vtk DataFile Version 2.0
some title
ASCII
DATASET RECTILINEAR_GRID
DIMENSIONS 2 3 4
X_COORDINATES 2 double
 0.00000000 1.00000000
Y_COORDINATES 3 double
 0.00000000 0.50000000 1.00000000
Z_COORDINATES 4 double
 0.00000000 0.33333333 0.66666667 1.00000000
POINT_DATA 24
SCALARS scalar_data double 1
LOOKUP_TABLE default
 0.00000000
 0.32719470
 0.61836980
 0.84147098
 0.00000000
 0.73618807
 1.39133206
 1.89330972
 0.00000000
 1.30877879
 2.47347921
 3.36588394
 0.00000000
 0.65438939
 1.23673961
 1.68294197
 0.00000000
 1.47237614
 2.78266411
 3.78661943
 0.00000000
 2.61755757
 4.94695842
 6.73176788
VECTORS vector_data double
 0.00000000 0.00000000 0.98158409
 0.32719470 0.81798674 0.92755470
 0.61836980 1.54592451 0.77141443
 0.84147098 2.10367746 0.66930355
 0.00000000 0.00000000 2.20856420
 0.73618807 0.98158409 2.08699809
 1.39133206 1.85510941 1.73568247
 1.89330972 2.52441295 1.50593298
 0.00000000 0.00000000 3.92633636
 1.30877879 1.14518144 3.71021882
 2.47347921 2.16429431 3

Let's see if we can open it with the `vtk` module:

In [7]:
import vtk
reader = vtk.vtkRectilinearGridReader()
reader.SetFileName("rectlin.vtk")
reader.ReadAllVectorsOn()
reader.ReadAllScalarsOn()
reader.Update()
data = reader.GetOutput()
print(data.GetDimensions())

(2, 3, 4)


In [8]:
point_data = data.GetPointData()
print(point_data)

vtkPointData (0x7fe047e105f0)
  Debug: Off
  Modified Time: 254
  Reference Count: 2
  Registered Events: 
    Registered Observers:
      vtkObserver (0x7fe047e10720)
        Event: 33
        EventName: ModifiedEvent
        Command: 0x7fe047e105b0
        Priority: 0
        Tag: 1
  Number Of Arrays: 2
  Array 0 name = scalar_data
  Array 1 name = vector_data
  Number Of Components: 4
  Number Of Tuples: 24
  Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
  Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
  Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
  Scalars: 
    Debug: Off
    Modified Time: 244
    Reference Count: 1
    Registered Events: (none)
    Name: scalar_data
    Data type: double
    Size: 25
    MaxId: 23
    NumberOfComponents: 1
    Information: 0x0
    Name: scalar_data
    Number Of Components: 1
    Number Of Tuples: 24
    Size: 25
    MaxId: 23
    LookupTable: (none)
  Vectors: 
    Debug: Off
    Modified Time: 251
    Reference Count: 1
    Registered Events: (none)
    Name: ve

In [9]:
scalars = point_data.GetArray(0)
print(scalars)

vtkDoubleArray (0x7fe04980ddb0)
  Debug: Off
  Modified Time: 244
  Reference Count: 2
  Registered Events: (none)
  Name: scalar_data
  Data type: double
  Size: 25
  MaxId: 23
  NumberOfComponents: 1
  Information: 0x0
  Name: scalar_data
  Number Of Components: 1
  Number Of Tuples: 24
  Size: 25
  MaxId: 23
  LookupTable: (none)




In [10]:
vtk.util.numpy_support.vtk_to_numpy(scalars)

AttributeError: module 'vtkmodules.all' has no attribute 'util'

The VTK Python interface isn't very well documented, but you can use this and Google to piece together custom processing code.  However, `pyevtk` gives us *some* of this  by default to produce a file in the newer XML (and binary) format:

In [ ]:
import pyevtk
pyevtk.hl.gridToVTK("rectlin2", x, y, z, pointData={'scalar_data': f})

As a second example, we'll create point-wise data that represents, perhaps, the location of several particles in a box at several successive time points.  This is one way to produce temporal viz.  We'll use `pyevt

In [ ]:
# Create points for 10 particles and 10 time steps. 
n_part, n_time = 10, 10
coords = np.zeros((n_time, 3, n_part))
coords[0, :, :] = np.random.rand(3, n_part)
# Quick 2-d mpl viz for identifying
for i in range(1, n_time):
    tmp = coords[i-1, :, :]+(np.random.rand(3, n_part)-0.5)/10
    tmp[tmp<0] = abs(tmp[tmp<0])   # reflection
    tmp[tmp>1] = abs(1-tmp[tmp>1]) # off walls
    coords[i, :, :] = tmp[:, :]
import matplotlib.pyplot as plt
for i in range(n_time):
    plt.plot(coords[i,0,:], coords[i,1,:], color='b', marker='o', mfc='w', ls='', ms=(1+i*2), alpha=0.5)
x0 = coords[0,0,:]
y0 = coords[0,1,:]
z0 = coords[0,2,:]
pyevtk.hl.pointsToVTK("./points", x0, y0, z0, data={'id': np.arange(0,10)})

We'll use these and other VTK files to vizualize things in VisIt (and, maybe Paraview and Mayavi).